In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from pylab import rcParams
rcParams['figure.figsize'] = (10, 7)

In [2]:
housing = pd.read_csv('housing_train_set.csv')

## Separate features and labels

In [3]:
# separate predictors and labels
housing_labels = housing.median_house_value.copy()
housing = housing.drop('median_house_value', axis=1)

## Feature combinations

In [4]:
housing['rooms_per_household'] = housing.total_rooms / housing.households
housing['bedrooms_per_room'] = housing.total_bedrooms / housing.total_rooms
housing['population_per_household'] = housing.population / housing.households

## Missing values

In [5]:
# what features has missing values

housing.isna().sum()

longitude                     0
latitude                      0
housing_median_age            0
total_rooms                   0
total_bedrooms              158
population                    0
households                    0
median_income                 0
ocean_proximity               0
rooms_per_household           0
bedrooms_per_room           158
population_per_household      0
dtype: int64

In [6]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

# drop categorical column sice imputer works only with numerical values
housing_num = housing.drop('ocean_proximity', axis=1)

X = imputer.fit_transform(housing_num)

housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

housing_tr.isna().sum()

longitude                   0
latitude                    0
housing_median_age          0
total_rooms                 0
total_bedrooms              0
population                  0
households                  0
median_income               0
rooms_per_household         0
bedrooms_per_room           0
population_per_household    0
dtype: int64

## Categorical features

In [9]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[['ocean_proximity']]

cat_encoder = OneHotEncoder()

housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

cat_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

## Transformation pipelines

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [13]:
from sklearn.compose import ColumnTransformer

num_features = list(housing_num)
cat_features = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', OneHotEncoder(), cat_features)
])

housing_prepared = full_pipeline.fit_transform(housing)

In [16]:
full_pipeline.named_transformers_['cat'].categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]